In [ ]:
# 환경변수에서 api키 가져오기
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

In [ ]:
# crewAI 라이브러리에서 필요한 클래스 가져오기
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI

#llm
llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-latest', temperature=0, api_key=GOOGLE_API_KEY)

In [ ]:
# web search tools

from langchain_community.tools.tavily_search import TavilySearchResults
search_tool = TavilySearchResults(api_key=TAVILY_API_KEY)

In [ ]:
# PDF Search Tool
from crewai_tools import PDFSearchTool

pdf_tool = PDFSearchTool(pdf='2024 서울관광안내서_KR.pdf')

pdf_tool2 = PDFSearchTool(pdf='2024 서울관광안내서_KR.pdf',
    config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini-1.5-pro-latest",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [ ]:
# Custom Tool - google map web crawling. 필요하다면 추가
from crewai_tools import tool

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

@tool("search_place_info")
def search_place_info(place_name: str) -> str:
    """
    Searches for a location on Google Maps.
    Returns operating hours, address, phone number, and fees.
    """


    # Chrome 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') 

    service = Service(ChromeDriverManager().install())

    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://www.google.com/maps/")
    driver.implicitly_wait(10)


    # 검색창에 입력하기
    input_search = driver.find_element(By.ID, 'searchboxinput')
    input_search.send_keys(place_name)
    driver.implicitly_wait(5)
    input_search.send_keys(Keys.RETURN)
    driver.implicitly_wait(5)
    time.sleep(3)


    # 장소가 여러 개 검색된 경우
    if f"{place_name}에대한검색결과" in driver.page_source.replace(" ", ""):
        search_results = driver.find_elements(By.CSS_SELECTOR, 'div > div > div > div > a')

        for n, item in enumerate(search_results):

            try:

                if place_name in item.get_attribute('aria-label'):
                    item.click()
                    driver.implicitly_wait(5)
                    time.sleep(3)
                    break

            except:
                pass


    # 장소 정보 가져오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    place_info = soup.find_all('div', attrs={'aria-label': place_name})
    place_info_text = "\n".join([info.text for info in place_info])

    driver.quit()

    return place_info_text

In [ ]:
place_name = "올림픽공원"

# Chrome 드라이버 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless') 

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.google.com/maps/")
driver.implicitly_wait(10)

In [ ]:
# 검색창에 입력하기
input_search = driver.find_element(By.ID, 'searchboxinput')
input_search.send_keys(place_name)
driver.implicitly_wait(5)
input_search.send_keys(Keys.RETURN)
driver.implicitly_wait(5)
time.sleep(3)

In [ ]:
# 장소가 여러 개 검색된 경우
if f"{place_name}에대한검색결과" in driver.page_source.replace(" ", ""):
    search_results = driver.find_elements(By.CSS_SELECTOR, 'div > div > div > div > a')

    for n, item in enumerate(search_results):

        try:

            if place_name in item.get_attribute('aria-label'):
                item.click()
                driver.implicitly_wait(5)
                time.sleep(3)
                break

        except:
            pass

In [ ]:
# 장소 정보 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')
place_info = soup.find_all('div', attrs={'aria-label': place_name})
place_info_text = "\n".join([info.text for info in place_info])

In [ ]:
print(place_info_text)

In [ ]:
driver.quit()

In [ ]:
# Agent
def tour_planning_expert(cities, interests):
  return Agent(
role='Tour Planning Expert',
goal=f'Select the best locations within {cities} based on weather, season, prices, and tourist preferences. Tourist preferences are {interests}',
backstory=f'An expert in analyzing local data to pick ideal destinations within {cities}',
verbose=True,
tools=[pdf_tool2, search_place_info],
allow_delegation=False,
llm=llm,
max_iter=3,
max_rpm=10,
)

def local_expert(cities, interests):
  return Agent(
role=f'{cities} Local Expert',
goal=f'Provide the BEST insights about the selected locations in {cities}',
backstory=f"""A knowledgeable local guide with extensive information
about {cities}'s attractions, customs, and hidden gems""",
verbose=True,
tools=[pdf_tool2, search_place_info],
allow_delegation=False,
llm=llm,
max_iter=3,
max_rpm=10,
)

# 여행 일정 짜줌
def travel_concierge(cities, interests):
  return Agent(
role=f'{cities} Custom Travel Concierge',
goal=f"""Create the most amazing travel itineraries for {cities} including budget and packing suggestions""",
backstory="""Specialist in Seoul travel planning and logistics with extensive experience""",
verbose=True,
ls=[search_tool],
allow_delegation=False,
llm=llm,
)

In [ ]:
# Tasks
def seoul_location_selection_task(agent, cities, interests):
  return Task(
    description=f'Identify the best locations within {cities} for visiting based on current weather, season, and user interests. User interests are {interests}.  Explain in Korean Hangul',
    agent=agent,
    expected_output=f'A list of recommended locations in {cities}, including reasons for each selection'
)


def seoul_local_insights_task(agent, cities, interests):
  return Task(
    description=f'Provide detailed insights and information about selected locations in {cities}, including attractions, customs, and hidden gems. Explain in Korean Hangul',
    agent=agent,
    expected_output=f'Comprehensive information about each location, including what to see, do, and eat'
)

def seoul_travel_itinerary_task(agent, cities, interests):
  return Task(
    description=f'Create a detailed travel itinerary for {cities} that includes budgeting, packing suggestions, accommodations, and transportation. Explain in Korean Hangul',
    agent=agent,
    expected_output=f'A complete travel plan for {cities}, including a day-by-day itinerary, budget estimates, and packing list. Key locations and place names should be provided in both English and Korean, such as "Museum(박물관)".'
)

In [ ]:
# Crew 생성  

trip_crew = Crew(
    agents=[tour_planning_expert, local_expert, travel_concierge],
    tasks=[seoul_location_selection_task, seoul_local_insights_task, seoul_travel_itinerary_task],
    process=Process.hierarchical,
    manager_llm=ChatGoogleGenerativeAI(model='gemini-1.5-pro-latest') 
)


In [ ]:
# Crew 생성
cities = input("What are the cities options you are interested in visiting?")
interests = input("What are some of your high level interests and hobbies?")


def trip_crew(cities,interests):
  tour_planning_expert = tour_planning_expert(cities, interests)
  local_expert = local_expert(cities, interests)
  travel_concierge = travel_concierge(cities, interests)

  seoul_location_selection_task = seoul_location_selection_task(tour_planning_expert, cities, interests)
  seoul_local_insights_task = seoul_local_insights_task(local_expert, cities, interests)
  seoul_travel_itinerary_task = seoul_travel_itinerary_task(travel_concierge, cities, interests)
  return Crew(
    agents=[tour_planning_expert, local_expert, travel_concierge],
    tasks=[seoul_location_selection_task, seoul_local_insights_task, seoul_travel_itinerary_task],
    process=Process.hierarchical,
    manager_llm=ChatGoogleGenerativeAI(model='gemini-1.5-pro-latest')
    )


In [ ]:
# Process 실행
trip_crew = trip_crew(cities, interests)
result = trip_crew.kickoff()

print(result)